In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

In [2]:
print(os.getcwd())
%cd ../../..

/Users/elizagoler/Documents/RecessionFertility/source/derived/county_gfrs
/Users/elizagoler/Documents/RecessionFertility


In [5]:
data_dir = "datastore/raw/nvss/data"
natality_dfs_68_73 = []

for year in range(1968, 1974):  # inclusive 1968 through 1978
    filename = f"natality{year}us.csv"
    filepath = os.path.join(data_dir, filename)
    if os.path.exists(filepath):
        df = pd.read_csv(filepath)
        df['datayear'] = year  # ensure datayear matches the file year
        natality_dfs_68_73.append(df)
    else:
        print(f"File {filename} does not exist.")

# Now natality_dfs_68_78 holds the 1968-1978 dfs with correct datayear column.


/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_18275/1710741445.py:8: DtypeWarning: Columns (9,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_18275/1710741445.py:8: DtypeWarning: Columns (6,7,8,9,10,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_18275/1710741445.py:8: DtypeWarning: Columns (6,7,8,9,10,54) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_18275/1710741445.py:8: DtypeWarning: Columns (1,2,6,7,8,9,10,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_18275/1710741445.py:8: DtypeWarning: Columns (1,2,6,7,

In [5]:
natality_68_73_concat = pd.concat(natality_dfs_68_73, ignore_index=True)

In [8]:
print(list(natality_68_73_concat.columns))

['datayear', 'rectype', 'restatus', 'stateres', 'cntyres', 'cityres', 'citrspop', 'smsares', 'metrores', 'frace', 'mrace', 'crace', 'crace2', 'crace3', 'csex', 'birmon', 'dmage', 'mage36', 'mage8', 'mage12', 'dlivord', 'livord9', 'livord3', 'birattnd', 'dgestat', 'gestat10', 'dbirwt', 'birwt12', 'birwt3', 'dlegit', 'dplural', 'plural2', 'statenat', 'cntynat', 'shipnum', 'reparea', 'divres', 'stsubres', 'divocc', 'stsubocc', 'mage15', 'mage7', 'mage6', 'nlbnl', 'nlbnd', 'nlbd', 'dtotord', 'totord9', 'livord8', 'livord7', 'livord6', 'flag77', 'dfage', 'fage11', 'birday', 'lmpmon', 'lmpday', 'lmpyr', 'gestat3', 'dmeduc', 'meduc14', 'meduc6', 'dfeduc', 'feduc14', 'legit3', 'monprec', 'mpre10', 'mpre6', 'llbmon', 'llbyr', 'disllb', 'isllb17', 'isllb10', 'isllb8', 'lfdmon', 'lfdyr', 'fdflg', 'dlfd', 'dterm', 'term9', 'outcome', 'congen', 'legitstrflg', 'educstrflg', 'lmprflg', 'monprerflg', 'llbrflg', 'lfdrflg', 'legitsmsarflg', 'educsmsarflg', 'congenrflg', 'legitstoflg', 'educstoflg', 'lmp

In [ ]:
natality_68_73_concat.head()
#datayear
#stateres
#cntyres
#cityres
#citrspop 

,datayear,rectype,restatus,stateres,cntyres,cityres,citrspop,smsares,metrores,frace,...,congenflg,mplbir,plural3,nprevis,nprevisrflg,nprevisoflg,recwt,nprev28,injuryrflg,injuryoflg
0,1968,1,1,1,1020,999,9,0,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1968,1,1,1,1001,999,9,0,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1968,1,1,1,1001,999,9,0,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1968,2,2,1,1026,999,9,0,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1968,1,1,1,1001,999,9,0,2,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Create a small version of natality_68_73_concat with the specified columns
natality_68_73_small = natality_68_73_concat[['datayear', 'stateres', 'cntyres']].copy()


In [ ]:
output_dir = "datastore/output/nvss"
n
natality_68_73_concat.to_parquet(os.path.join(output_dir, "nvss_68_79.parquet"), index=False)

: 

In [ ]:
data_dir = "datastore/raw/nvss/data"
natality_dfs_74_78 = []

for year in range(1974, 1979):  
    filename = f"natality{year}us.csv"
    filepath = os.path.join(data_dir, filename)
    if os.path.exists(filepath):
        df = pd.read_csv(filepath)
        df['datayear'] = year  
        natality_dfs_74_78.append(df)
    else:
        print(f"File {filename} does not exist.")

In [31]:
# Make a copy of the list and keep only relevant columns in each DataFrame
# Convert cntyres and cityres to string to avoid type conversion issues
natality_small_dfs_68_78 = [df[['datayear', 'cntyres', 'cityres']].copy() for df in natality_dfs_68_78]
for df in natality_small_dfs_68_78:
    df['cntyres'] = df['cntyres'].astype(str)
    df['cityres'] = df['cityres'].astype(str)

# Concatenate all smaller DataFrames into a single DataFrame
nvss_68_79_small = pd.concat(natality_small_dfs_68_78, ignore_index=True)

# Ensure the output directory exists
output_dir = "datastore/output/nvss"
os.makedirs(output_dir, exist_ok=True)

# Save the concatenated dataframe to Parquet
nvss_68_79_small.to_parquet(os.path.join(output_dir, "nvss_68_79.parquet"), index=False)

In [4]:
output_dir = "datastore/output/nvss"
pd.read_parquet(os.path.join(output_dir, "nvss_68_79.parquet"))

,datayear,cntyres,cityres
0,1968,1020,999
1,1968,1001,999
2,1968,1001,999
3,1968,1026,999
4,1968,1001,999
...,...,...,...
23153992,1978,51004,999
23153993,1978,51014,999
23153994,1978,51018,999
23153995,1978,51007,999


In [5]:
data_dir = "datastore/raw/nvss/data"
natality_dfs_79_89= []

for year in range(1979, 1989):  # inclusive 1979 through 1989
    filename = f"natality{year}us.csv"
    filepath = os.path.join(data_dir, filename)
    if os.path.exists(filepath):
        df = pd.read_csv(filepath)
        df['datayear'] = year  # ensure datayear matches the file year
        natality_dfs_79_89.append(df)
    else:
        print(f"File {filename} does not exist.")


/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_77145/1448024086.py:8: DtypeWarning: Columns (1,6,7,8,9,10,54) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_77145/1448024086.py:8: DtypeWarning: Columns (1,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_77145/1448024086.py:8: DtypeWarning: Columns (1,6,7,8,9,10,117) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_77145/1448024086.py:8: DtypeWarning: Columns (6,7,8,9,10,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_77145/1448024086.py:8: DtypeWarning: Columns 

In [ ]:
# Make a copy of the list and keep only relevant columns in each DataFrame
# Convert cntyres and cityres to string to avoid type conversion issues
natality_small_dfs_79_89 = [df[['datayear', 'cntyres', 'cityres']].copy() for df in natality_dfs_79_89]
for df in natality_small_dfs_79_89:
    df['cntyres'] = df['cntyres'].astype(str)
    df['cityres'] = df['cityres'].astype(str)

# Concatenate all smaller DataFrames into a single DataFrame
nvss_79_89_small = pd.concat(natality_small_dfs_79_89, ignore_index=True)

# Ensure the output directory exists
output_dir = "datastore/output/nvss"
os.makedirs(output_dir, exist_ok=True)

# Save the concatenated dataframe to Parquet
nvss_79_89_small.to_parquet(os.path.join(output_dir, "nvss_79_88.parquet"), index=False)

In [10]:
pd.read_parquet(os.path.join(output_dir, "nvss_79_88.parquet"))

,datayear,cntyres,cityres
0,1979,01002,999
1,1979,01002,999
2,1979,01002,999
3,1979,01002,999
4,1979,01002,999
...,...,...,...
35142106,1988,51011,999
35142107,1988,51011,999
35142108,1988,51011,999
35142109,1988,51015,999


In [ ]:
# Make a copy of the list and keep only relevant columns in each DataFrame
# Convert cntyres and cityres to string to avoid type conversion issues
natality_small_dfs_68_78 = [df[['datayear', 'cntyres', 'cityres']].copy() for df in natality_dfs_68_78]
for df in natality_small_dfs_68_78:
    df['cntyres'] = df['cntyres'].astype(str)
    df['cityres'] = df['cityres'].astype(str)

# Concatenate all smaller DataFrames into a single DataFrame
nvss_68_79_small = pd.concat(natality_small_dfs_68_78, ignore_index=True)

# Ensure the output directory exists
output_dir = "datastore/output/nvss"
os.makedirs(output_dir, exist_ok=True)

# Save the concatenated dataframe to Parquet
nvss_68_79_small.to_parquet(os.path.join(output_dir, "nvss_68_79.parquet"), index=False)

In [3]:
data_dir = "datastore/raw/nvss/data"
natality_dfs_89_93 = []

for year in range(1989, 1994):  # inclusive 1989 through 1999
    filename = f"natality{year}us.csv"
    filepath = os.path.join(data_dir, filename)
    if os.path.exists(filepath):
        df = pd.read_csv(filepath)
        df['datayear'] = year  # ensure datayear matches the file year
        natality_dfs_89_93.append(df)
    else:
        print(f"File {filename} does not exist.")

/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_85249/464400410.py:8: DtypeWarning: Columns (21,22,23,24,28,203) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_85249/464400410.py:8: DtypeWarning: Columns (21,22,23,24,29,204,205) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_85249/464400410.py:8: DtypeWarning: Columns (21,22,23,24,29,204,205) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_85249/464400410.py:8: DtypeWarning: Columns (21,22,23,24,29,206,207) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_85249/464400410.py:8:

In [4]:
len(natality_dfs_89_93)

5

In [5]:
# Make a copy of the list and keep only relevant columns in each DataFrame
# Convert cntyres and cityres to string to avoid type conversion issues
natality_small_dfs_89_93 = [df[['datayear', 'cntyres', 'cityres']].copy() for df in natality_dfs_89_93]
for df in natality_small_dfs_89_93:
    df['cntyres'] = df['cntyres'].astype(str)
    df['cityres'] = df['cityres'].astype(str)

# Concatenate all smaller DataFrames into a single DataFrame
nvss_89_93_small = pd.concat(natality_small_dfs_89_93, ignore_index=True)

# Ensure the output directory exists
output_dir = "datastore/output/nvss"
os.makedirs(output_dir, exist_ok=True)

output_dir = "datastore/output/nvss"
nvss_89_93_small.to_parquet(os.path.join(output_dir, "nvss_89_93.parquet"), index=False)

In [2]:
data_dir = "datastore/raw/nvss/data"
natality_dfs_94_95 = []

for year in range(1994, 1996):  # inclusive 1989 through 1999
    filename = f"natality{year}us.csv"
    filepath = os.path.join(data_dir, filename)
    if os.path.exists(filepath):
        df = pd.read_csv(filepath)
        df['datayear'] = year  # ensure datayear matches the file year
        natality_dfs_94_95.append(df)
    else:
        print(f"File {filename} does not exist.")

NameError: name 'os' is not defined

In [6]:
import os
import pandas as pd

data_dir = "datastore/raw/nvss/data"
files = [
    os.path.join(data_dir, f)
    for f in os.listdir(data_dir)
    if os.path.isfile(os.path.join(data_dir, f)) and f.lower().endswith(".csv")
]

for file in files:
    try:
        df = pd.read_csv(file, nrows=2)
        print(f"\nColumns in {os.path.basename(file)}:")
        print(df.columns)
    except Exception as e:
        print(f"Could not read {file}: {e}")



Columns in natality1983us.csv:
Index(['datayear', 'stresexp', 'reparea', 'rectype', 'restatus', 'stateres',
       'cntyres', 'cityres', 'citrspop', 'smsares',
       ...
       'regnocc', 'stoccfip', 'cntocfip', 'stresfip', 'cntyrfip', 'smsarfip',
       'recwt', 'nprev28', 'nprev12', 'stnatexp'],
      dtype='object', length=129)

Columns in natality2010us.csv:
Index(['revision', 'dob_yy', 'dob_mm', 'dob_tt', 'dob_wk', 'oterr',
       'ocntyfips', 'ocntypop', 'bfacil', 'ubfacil',
       ...
       'f_uld_precip', 'f_uld_breech', 'f_u_forcep', 'f_u_vac', 'f_uca_anen',
       'f_uca_spina', 'f_uca_omphalo', 'f_uca_cleftlp', 'f_uca_hernia',
       'f_uca_downs'],
      dtype='object', length=269)

Columns in natality1981us.csv:
Index(['datayear', 'shipnum', 'reparea', 'rectype', 'restatus', 'stateres',
       'cntyres', 'cityres', 'citrspop', 'smsares',
       ...
       'omaps', 'omapsr', 'fmaps', 'fmapsr', 'origm', 'origf', 'smsarfip',
       'recwt', 'nprev28', 'nprev12'],
      dty

In [26]:
# Check if files for 2006-2010 have any columns with 'weight' or 'wt' in their names
# And then which dfs have columns with string 'pop' in them
import re

years_of_interest = [2006, 2007, 2008, 2009, 2010]
weight_columns_by_year = {}
pop_columns_by_file = {}

for file in files:
    basename = os.path.basename(file)
    match = re.search(r"(\d{4})", basename)
    try:
        df = pd.read_csv(file, nrows=2)
        # check for weight/wt cols for 2006-2010
        if match:
            year = int(match.group(1))
            if year in years_of_interest:
                columns_with_weight = [col for col in df.columns if 'weight' in col.lower() or 'wt' in col.lower()]
                weight_columns_by_year[year] = columns_with_weight
        # check for pop cols in all files
        columns_with_pop = [col for col in df.columns if 'pop' in col.lower()]
        if columns_with_pop:
            pop_columns_by_file[basename] = columns_with_pop
    except Exception as e:
        if match and int(match.group(1)) in years_of_interest:
            weight_columns_by_year[int(match.group(1))] = f"Error: {e}"
        pop_columns_by_file[basename] = f"Error: {e}"

for year in sorted(weight_columns_by_year):
    print(f"Year {year}: columns with 'weight' or 'wt' -> {weight_columns_by_year[year]}")

print("\nFiles with columns containing 'pop':")
for fname, cols in pop_columns_by_file.items():
    print(f"{fname}: {cols}")


Year 2006: columns with 'weight' or 'wt' -> ['wtgain', 'wtgain_rec', 'dbwt', 'bwtr12', 'bwtr4', 'f_wtgain']
Year 2007: columns with 'weight' or 'wt' -> ['wtgain', 'wtgain_rec', 'dbwt', 'bwtr12', 'bwtr4', 'f_wtgain']
Year 2008: columns with 'weight' or 'wt' -> ['wtgain', 'wtgain_rec', 'dbwt', 'bwtr12', 'bwtr4', 'f_wtgain']
Year 2009: columns with 'weight' or 'wt' -> ['wtgain', 'wtgain_rec', 'dbwt', 'bwtr12', 'bwtr4', 'f_wtgain']
Year 2010: columns with 'weight' or 'wt' -> ['wtgain', 'wtgain_rec', 'dbwt', 'bwtr12', 'bwtr4', 'f_wtgain']

Files with columns containing 'pop':
natality1983us.csv: ['citrspop']
natality2010us.csv: ['ocntypop', 'rcnty_pop']
natality1981us.csv: ['citrspop']
natality1998us.csv: ['cntocpop', 'citrspop', 'cntrspop', 'popsmas']
natality2009us.csv: ['ocntypop', 'rcnty_pop']
natality1985us.csv: ['citrspop']
natality1978us.csv: ['citrspop']
natality1987us.csv: ['citrspop']
natality1980us.csv: ['citrspop']
natality2008us.csv: ['ocntypop', 'rcnty_pop']
natality1999us.csv

In [7]:
count = 0
for file in files:
    try:
        df = pd.read_csv(file, nrows=2)
        if "cntyrfip" in df.columns:
            count += 1
    except Exception as e:
        continue
print(count)


21


In [8]:
print(len(files))


43


In [9]:
datayear_count = 0
for file in files:
    try:
        df = pd.read_csv(file, nrows=2)
        if "datayear" in df.columns:
            datayear_count += 1
    except Exception as e:
        continue
print(datayear_count)


35


In [10]:
# For files that don't have column 'datayear', check if they have any column containing the string 'year'
no_datayear_files = []
for file in files:
    try:
        df = pd.read_csv(file, nrows=2)
        if "datayear" not in df.columns:
            no_datayear_files.append((file, df.columns))
    except Exception as e:
        continue

for file, columns in no_datayear_files:
    year_columns = [col for col in columns if "year" in col.lower()]
    print(f"{os.path.basename(file)} - Columns with 'year': {year_columns}")


natality2010us.csv - Columns with 'year': []
natality2009us.csv - Columns with 'year': []
natality2008us.csv - Columns with 'year': []
natality2006us.csv - Columns with 'year': []
natality2004us.csv - Columns with 'year': []
natality2003us.csv - Columns with 'year': []
natality2005us.csv - Columns with 'year': []
natality2007us.csv - Columns with 'year': []


In [14]:
state_columns = set()
for file in files:
    try:
        df = pd.read_csv(file, nrows=2)
        for col in df.columns:
            if "state" in col.lower():
                state_columns.add(col)
    except Exception as e:
        continue
print(len(state_columns))
print("State-related columns found:", state_columns)


8
State-related columns found: {'mrstate', 'stateres', 'xmrstate', 'ostate', 'mbstate_rec', 'umbstate', 'statenat', 'xostate'}


In [19]:
cntyres_count = 0
for file in files:
    try:
        df = pd.read_csv(file, nrows=2)
        if "cntyres" in df.columns:
            cntyres_count += 1
    except Exception as e:
        continue
print(cntyres_count)


35


In [20]:
# Check if files for 2003-2010 have any columns that include the string 'fip'
years_to_check = [2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003]
files_to_check = [file for file in files if any(str(year) in os.path.basename(file) for year in years_to_check)]

for file in files_to_check:
    try:
        df = pd.read_csv(file, nrows=2)
        fip_columns = [col for col in df.columns if "cntyres" in col.lower()]
        print(f"{os.path.basename(file)} - Columns containing 'fip': {fip_columns}")
    except Exception as e:
        continue


natality2010us.csv - Columns containing 'fip': []
natality2009us.csv - Columns containing 'fip': []
natality2008us.csv - Columns containing 'fip': []
natality2006us.csv - Columns containing 'fip': []
natality2004us.csv - Columns containing 'fip': []
natality2003us.csv - Columns containing 'fip': []
natality2005us.csv - Columns containing 'fip': []
natality2007us.csv - Columns containing 'fip': []


**Include:
- datayear (except for 2003-2010 add a column 'datayear' with the corresponding year)
- cntyrfip (available for 21 of the years)
- stateres (35 of the files have)
- cntyres (35 of the files have)
- ocntyfips

**Eight of the dataframes that do not have fip also do not have stateres or cntyres but they are after 1995, so I don't need them**